# Get Masked Betas for each participant

First, imports and paths:

In [8]:
# Imports and setup paths
import nilearn
from os.path import join
from os import listdir
import numpy as np
import pandas as pd

steph_dir = '/imaging/astle/users/su01/RED'
perm_dir = join(steph_dir, '3 Permutations_SnPM/Outputs/Go/1 Sad vs Happy')
group_mask_file= join(perm_dir, 'SadvsHappy_neg_SnPM_filtered.nii')

read in the .nii file from the group level as data

Reading in the data gives us a numpy array, which is much more flexible than niilearns own formats

This is a 3D array representing voxels, nan values represent no significance, anything not a nan is a significant beta


In [2]:
group_mask = nilearn.image.get_data(group_mask_file)


 we now convert this image of NaN's and numbers into a boolean array, which we can later use as an index
 Numpy allows boolean indexing, so we can just use our mask as an array to blank out other files
 See: https://numpy.org/devdocs/reference/arrays.indexing.html#boolean-array-indexing

 we use numpy is nan, which returns True if a number is NaN -- we want to reverse this, so we put the '~' preceding


In [3]:
group_boolean_mask = ~np.isnan(group_mask) # get a mask of true and false for if that voxel was significant

now we want to loop through the lower level files and get a beta value for each participant using this mask

First, set up paths to the 1st level directories and contrast names.

Then make a list of all the subject folders

In [4]:
first_level_directory = join(steph_dir, '1stlevel_output_go', '1stLevel')
cont_name = 'con_0004.nii' # make sure this matches the GLM contrast ie. 'Happy Go > Sad Go'
sublist = listdir(first_level_directory) # list all directories in this folder

Now loop through the subject folders using enumerate

We then mask these lower level statsitics and can use them to get a summary stat for each child

In [5]:
beta_vals = [] # our beta value list holder
for i, sub_dir in enumerate(sublist):
    this_beta = nilearn.image.get_data(join(first_level_directory, sub_dir, cont_name))# read in the betas from this participant, again as a numpy array
    this_masked_vals = this_beta[group_boolean_mask] # mask them using the boolean mask
    this_masked_vals = np.abs(this_masked_vals) # take absolute values (for ease)
    this_beta_stat = np.mean(this_masked_vals) # take sum (but you can use average - np.mean(), peak np.max() or whatever function you'd like)
    beta_vals.append((sub_dir, this_beta_stat)) # append this participants value along with their ID


now we have the summed/averaged (depending) activity for each participant, masked by the group level contrast

If significant, we can also go back and get the masked values only and look at which voxels most predict the covariates
but for now these are pretty good as is

In [6]:
print(beta_vals)

[('sub-021', 1.3219343), ('sub-045', 0.85171926), ('sub-019', 0.2266045), ('sub-013', 0.29642582), ('sub-064', 0.20871297), ('sub-007', 0.39665985), ('sub-077', 0.990474), ('sub-035', 1.398368), ('sub-070', 0.38727096), ('sub-063', 0.6934938), ('sub-051', 0.6706148), ('sub-048', 3.2704482), ('sub-014', 0.7282375), ('sub-055', 0.904426), ('sub-010', 1.143196), ('sub-009', 0.3485318), ('sub-003', 0.24464726), ('sub-073', 0.32062444), ('sub-025', 0.7607588), ('sub-041', 0.46935704), ('sub-060', 0.8355631), ('sub-015', 0.20245661), ('sub-058', 0.6856449), ('sub-027', 0.65726626), ('sub-062', 0.25794914), ('sub-017', 0.12783174), ('sub-006', 0.32926667), ('sub-044', 0.32451108), ('sub-039', 0.2546552), ('sub-018', 0.37408143), ('sub-012', 0.30326536), ('sub-024', 0.45470142), ('sub-016', 0.96895236), ('sub-053', 0.24564704), ('sub-072', 0.21654549), ('sub-037', 0.4674758), ('sub-005', 0.23174523), ('sub-047', 0.5709), ('sub-008', 0.5453956), ('sub-075', 0.42705095), ('sub-002', 0.36208087),

In [18]:
# Make into dataframe with columns
beta_df = pd.DataFrame(np.array(beta_vals), columns = ['sub_id', 'averaged_betas'])
beta_df.averaged_betas = beta_df.averaged_betas.astype('float') #to make into numeric values
beta_df = beta_df.sort_values('sub_id') #sort in ascending order by sub-id

In [20]:
# write into csv (ownCloud directory for merging purposes)
beta_df.to_csv(steph_dir + '/Beta outputs/happygo_sadgo_betas.csv', index = False)

## Final analysis steps
Can look at any of the covariates

May do this on local computer so will need to change some directories and paths if that is the case

In [9]:
# load BIDS2RED and merge to get RED ids for data
bid2red = pd.read_excel('/imaging/astle/users/ai05/RED/RED_fmri_finalids.xlsx')
main_df = pd.merge(b_df, bid2red, on='sub_id')

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [ ]:
#load in all metadata
META_PATH = join('/imaging', 'astle','users','ai05', 'RED', 'RED_MEG', 'worms', 'Combined3.csv')
meta_df = pd.read_csv(META_PATH)
meta_df.rename(columns={'Alex_ID': 'red_id'},inplace=True)
main_df['red_id'] = main_df['red_id'].astype(str)
with_data = pd.merge(main_df, meta_df, on='red_id')


#%% Now do some analysis -- this can be whatever covariate you want really
plt.close('all')
sns.regplot(y='summed_betas', x='WASI_Mat', data=with_data)
plt.show()